In [1]:
# Import các thư viện cần thiết
import numpy as np
import cv2
import pandas as pd
import os
import pickle
from google.colab import drive
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split

In [2]:
# Connect Google Drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/test_opencv/'
# Hiển thị tất cả thư mục hoặc file
os.listdir(path)

Mounted at /content/drive


['image_40.jpg',
 'image_60.jpg',
 'image_80.jpg',
 'image_100.jpg',
 '27.ipynb',
 '30.ipynb',
 'FaceImage',
 'models',
 'data_face_features.pickle',
 '31c_data_preprocessing.ipynb',
 '29.ipynb',
 '28.ipynb',
 'Untitled0.ipynb',
 '32.ipynb']

In [3]:
# Load data from pickle file
data = pickle.load(open("/content/drive/MyDrive/test_opencv/data_face_features.pickle", mode="rb"))

In [4]:
x = np.array(data['data'])
y = np.array(data['label'])

print(x)
print(y)

[[[ 0.03563213  0.14579633 -0.07474371 ...  0.10712294  0.05235191
    0.01346222]]

 [[ 0.01891751  0.07340851 -0.06299383 ...  0.0674775  -0.07994623
   -0.04636956]]

 [[ 0.03980485  0.15056151 -0.00706803 ...  0.12454392  0.04065556
   -0.02980149]]

 ...

 [[ 0.10115854  0.26688126 -0.05408768 ... -0.08945148  0.16420038
    0.0283456 ]]

 [[ 0.1297804   0.09853813 -0.07152011 ... -0.02761066  0.08419115
   -0.00754967]]

 [[ 0.06435601  0.00615203 -0.04063413 ...  0.11231432  0.06116884
    0.05675695]]]
['1721031660_LeQuangQuocDat' '1721031660_LeQuangQuocDat'
 '1721031660_LeQuangQuocDat' ... '162000186_NguyenVanHuong'
 '162000186_NguyenVanHuong' '162000186_NguyenVanHuong']


In [5]:
x.shape

(4699, 1, 128)

In [6]:
y.shape

(4699,)

In [7]:
x = x.reshape(-1, 128)

In [8]:
x.shape

(4699, 128)

In [9]:
# chia nhỏ dữ liệu
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.9, random_state=1)

In [10]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((4229, 128), (470, 128), (4229,), (470,))

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [12]:
def get_report(model, x_train, y_train, x_test, y_test):
  y_pred_train = model.predict(x_train)
  y_pred_test = model.predict(x_test)

  # accuracy score
  acc_train = accuracy_score(y_train, y_pred_train)
  acc_test = accuracy_score(y_test, y_pred_test)

  # f1 score
  f1_score_train = f1_score(y_train, y_pred_train, average="macro")
  f1_score_test = f1_score(y_test, y_pred_test, average="macro")

  # print
  print('Accurency Train = %0.2f'%acc_train)
  print('Accurency Test = %0.2f'%acc_test)
  print('F1 Score Train = %0.2f'%f1_score_train)
  print('F1 Score Test = %0.2f'%f1_score_test)

In [13]:
# Logistic regression
model_logistic = LogisticRegression()
model_logistic.fit(x_train, y_train) # training

LogisticRegression()

In [14]:
# Support Vector Machines
model_svc = SVC(probability=True)
model_svc.fit(x_train, y_train)

SVC(probability=True)

In [15]:
get_report(model_svc, x_train, y_train, x_test, y_test)

Accurency Train = 0.91
Accurency Test = 0.83
F1 Score Train = 0.84
F1 Score Test = 0.77


In [16]:
# Random forest
model_rf = RandomForestClassifier(n_estimators=20,)
model_rf.fit(x_train, y_train)

RandomForestClassifier(n_estimators=20)

In [17]:
get_report(model_rf, x_train, y_train, x_test, y_test)

Accurency Train = 1.00
Accurency Test = 0.75
F1 Score Train = 1.00
F1 Score Test = 0.65


In [18]:
# Voting Classifier
model_voting = VotingClassifier(estimators=[
    ('logistic', LogisticRegression()),
    ('svm', SVC(probability=True)),
    ('rf', RandomForestClassifier(n_estimators=20,)),
], voting='soft', weights=[2,3,1])

In [19]:
model_voting.fit(x_train, y_train)

VotingClassifier(estimators=[('logistic', LogisticRegression()),
                             ('svm', SVC(probability=True)),
                             ('rf', RandomForestClassifier(n_estimators=20))],
                 voting='soft', weights=[2, 3, 1])

In [20]:
get_report(model_voting, x_train, y_train, x_test, y_test)

Accurency Train = 0.98
Accurency Test = 0.84
F1 Score Train = 0.97
F1 Score Test = 0.78


In [21]:
# 3. Parameter Tuning
from sklearn.model_selection import GridSearchCV

model_grid = GridSearchCV(model_voting, param_grid={
    'svm__C': [3,5,7,10],
    'svm__gamma': [0.1, 0.3, 0.5],
    'rf__n_estimators': [5, 10,20],
    'rf__max_depth': [3, 5, 7],
    'voting': ['soft', 'hard']
}, scoring='accuracy', cv=3, n_jobs=1, verbose=2)

In [22]:
model_grid.fit(x_train, y_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   8.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   6.7s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   7.8s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   6.1s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   7.9s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   6.9s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   7.9s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   6.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   7.1s
[

GridSearchCV(cv=3,
             estimator=VotingClassifier(estimators=[('logistic',
                                                     LogisticRegression()),
                                                    ('svm',
                                                     SVC(probability=True)),
                                                    ('rf',
                                                     RandomForestClassifier(n_estimators=20))],
                                        voting='soft', weights=[2, 3, 1]),
             n_jobs=1,
             param_grid={'rf__max_depth': [3, 5, 7],
                         'rf__n_estimators': [5, 10, 20],
                         'svm__C': [3, 5, 7, 10], 'svm__gamma': [0.1, 0.3, 0.5],
                         'voting': ['soft', 'hard']},
             scoring='accuracy', verbose=2)

In [23]:
model_best_estimator = model_grid.best_estimator_

In [24]:
model_grid.best_score_

0.8458273141087269

In [25]:
# 4. Save Model
pickle.dump(model_best_estimator, open("/content/drive/MyDrive/test_opencv/ml_face_person_identity.pkl", mode='wb'))